In [103]:
import dataclasses
import math
from typing import List


@dataclasses.dataclass
class Blueprint:
    index: int
    ore_robot_ore: int
    clay_robot_ore: int
    obsidian_robot_ore: int
    obsidian_robot_clay: int
    geode_robot_ore: int
    geode_robot_obsidian: int


@dataclasses.dataclass
class State:
    minute: int
    ore: int
    clay: int
    obsidian: int
    geode: int
    ore_robots: int
    clay_robots: int
    obsidian_robots: int
    geode_robots: int
    history: str


def run_blueprint(blueprint: Blueprint, duration: int):   
    open_states: List[State] = [State(0, 0, 0, 0, 0, 1, 0, 0, 0, '')]
    max_state = open_states[0]

    while open_states:
        state = open_states.pop()

        if state.minute > duration:
            continue
            
        if state.geode > max_state.geode:
            max_state = state
        
        if state.minute == duration:
            continue
            
        # path for buying an ore robot
        if state.ore_robots < max(blueprint.ore_robot_ore, blueprint.clay_robot_ore, blueprint.obsidian_robot_ore, blueprint.geode_robot_ore):
            completion_duration = max(0, math.ceil((blueprint.ore_robot_ore - state.ore) / state.ore_robots)) + 1
            completion_duration = duration - state.minute if state.minute + completion_duration > duration else completion_duration
            ore_robot_state = dataclasses.replace(state,
                minute=state.minute + completion_duration,
                ore=state.ore + completion_duration * state.ore_robots - blueprint.ore_robot_ore,
                clay=state.clay + completion_duration * state.clay_robots,
                obsidian=state.obsidian + completion_duration * state.obsidian_robots,
                geode=state.geode + completion_duration * state.geode_robots,
                ore_robots=state.ore_robots + 1,
                history=state.history + f'@{state.minute + completion_duration}:ore_robot '
            )

            open_states.append(ore_robot_state)

        # path for buying an clay robot
        if state.clay_robots < blueprint.obsidian_robot_clay:
            completion_duration = max(0, math.ceil((blueprint.clay_robot_ore - state.ore) / state.ore_robots)) + 1
            completion_duration = duration - state.minute if state.minute + completion_duration > duration else completion_duration
            clay_robot_state = dataclasses.replace(state,
                minute=state.minute + completion_duration,
                ore=state.ore + completion_duration * state.ore_robots - blueprint.clay_robot_ore,
                clay=state.clay + completion_duration * state.clay_robots,
                obsidian=state.obsidian + completion_duration * state.obsidian_robots,
                geode=state.geode + completion_duration * state.geode_robots,
                clay_robots=state.clay_robots + 1,
                history=state.history + f'@{state.minute + completion_duration}:clay_robot '
            )

            open_states.append(clay_robot_state)

        # path for buying an obsidian robot
        if state.clay_robots and state.obsidian_robots < blueprint.geode_robot_obsidian:
            completion_duration = max(0, math.ceil((blueprint.obsidian_robot_ore - state.ore) / state.ore_robots), math.ceil((blueprint.obsidian_robot_clay - state.clay) / state.clay_robots)) + 1
            completion_duration = duration - state.minute if state.minute + completion_duration > duration else completion_duration
            obsidian_robot_state = dataclasses.replace(state,
                minute=state.minute + completion_duration,
                ore=state.ore + completion_duration * state.ore_robots - blueprint.obsidian_robot_ore,
                clay=state.clay + completion_duration * state.clay_robots - blueprint.obsidian_robot_clay,
                obsidian=state.obsidian + completion_duration * state.obsidian_robots,
                geode=state.geode + completion_duration * state.geode_robots,
                obsidian_robots=state.obsidian_robots + 1,
            history=state.history + f'@{state.minute + completion_duration}:obsidian_robot '
            )

            open_states.append(obsidian_robot_state)
        
        # path for buying an geode robot
        if state.obsidian_robots:
            completion_duration = max(0, math.ceil((blueprint.geode_robot_ore - state.ore) / state.ore_robots), math.ceil((blueprint.geode_robot_obsidian - state.obsidian) / state.obsidian_robots)) + 1
            completion_duration = duration - state.minute if state.minute + completion_duration > duration else completion_duration
            geode_robot_state = dataclasses.replace(state,
                minute=state.minute + completion_duration,
                ore=state.ore + completion_duration * state.ore_robots - blueprint.geode_robot_ore,
                clay=state.clay + completion_duration * state.clay_robots,
                obsidian=state.obsidian + completion_duration * state.obsidian_robots - blueprint.geode_robot_obsidian,
                geode=state.geode + completion_duration * state.geode_robots,
                geode_robots=state.geode_robots + 1,
                history=state.history + f'@{state.minute + completion_duration}:geode_robot '
            )

            open_states.append(geode_robot_state)

    return max_state

In [104]:
import re
file_data = open('day19.data', 'r').read().splitlines()

blueprints = [Blueprint(*(int(v) for v in re.findall(r'Blueprint (\d+): Each ore robot costs (\d+) ore. Each clay robot costs (\d+) ore. Each obsidian robot costs (\d+) ore and (\d+) clay. Each geode robot costs (\d+) ore and (\d+) obsidian.', line)[0])) for line in file_data]
results = [(b.index, run_blueprint(b, 24)) for b in blueprints]
sum(r[0] * r[1].geode for r in results)

1616

In [106]:
run_blueprint(blueprints[0], 32).geode * run_blueprint(blueprints[1], 32).geode * run_blueprint(blueprints[2], 32).geode

8990